# TODO
- vP is not allow to be 0
- add everything to dictionaries
- attacker only works with dict
- optimise
- try catch when non invertible prime
- t must not be prime --> optimise
- when there is no R large/ small en
- when delta of bounds is small then the possibility of a new generation of p is high 
- when p is smaller than there is is higher chance of success



### Import

In [882]:
import sympy
import random
import coloredlogs, logging
coloredlogs.install(fmt='%(levelname)s %(message)s')

In [883]:
logger = logging.getLogger()
logger.setLevel(logging.NOTSET)

In [884]:
MIN_PRIME = 0
MAX_PRIME = 10
OPT_RANGE_REDUCTION = 0

MAX_ITERS = 10



msg = 12345678900987654321

In [885]:
def genPrime():
    return sympy.randprime(pow(2,MIN_PRIME), pow(2,MAX_PRIME))

def genSlightlySmallerPrime():
    return sympy.randprime(pow(2,MIN_PRIME), pow(2,MAX_PRIME-OPT_RANGE_REDUCTION))

def genPrimeWithLimit(lb, ub):
    return sympy.randprime(lb, ub)

def genSmallPrime():
    return sympy.randprime(pow(2,MIN_PRIME/2), pow(2,MAX_PRIME/2))

def fermatPrime():
    i = random.randint(0,4)
    return pow(2, pow(2, i)) + 1

def mmi(E, phiN):
    return pow(E, -1, phiN)

def calcN(P, Q):
    return P*Q

def calcPhiN(P, Q):
    return (P-1) * (Q-1)

def toBinary(dec):
    return "{0:b}".format(dec)

def toDecimal(bin):
    return int(bin, 2)

def concatenateInBinary(prev, tail):
    prev = toBinary(prev)
    tail = toBinary(tail)

    prev = "0"*(MAX_PRIME-len(prev)) + prev
    tail = "0"*(MAX_PRIME-len(tail)) + tail

    return toDecimal(prev + tail)

def splitInBinary(bin):
    bin = toBinary(bin)
    bin = "0"*(MAX_PRIME*2-len(bin)) + bin
    prev = bin[:len(bin)//2]
    tail = bin[len(bin)//2:]
    return(toDecimal(prev), toDecimal(tail))



# Voraussetzungen

In [886]:
def createAttackerKey():
    a_P = genSmallPrime()
    a_Q= genSmallPrime()


    a_PhiN = calcPhiN(a_P, a_Q)
    a_N = calcN(a_P, a_Q)

    a_E = fermatPrime()
    a_D = mmi(a_E, a_PhiN)

    return {
        "P" : a_P,
        "Q" : a_Q,
        "PhiN" : a_PhiN,
        "N" : a_N,
        "E" : a_E,
        "D" : a_D
    }

Attacker = {}

created = False
while not created:
    try:
        Attacker = createAttackerKey()
        created = True
    except ValueError:
            print("Trying to generate Key failed. Trying again ...")

print(Attacker)

{'P': 2, 'Q': 5, 'PhiN': 4, 'N': 10, 'E': 3, 'D': 3}


# Generierung und Verschlüsselung

## Schritt 1, 2 und 3

In [887]:
r_too_large = 0
bounds_too_close = 0

# Do I have a problem when R = 0?
# This has to be optimized
# BAD Inspiration from https://www.cryptool.org/assets/ctp/documents/Kleptografie_kes_4-2010.pdf
User = {}

valid = False
while not valid:
    # Schritt 1
    P = genSlightlySmallerPrime()
    vP = pow(P, Attacker["E"] ,Attacker["N"])

    # Schritt 2
    t = genPrime() 
    tempN = concatenateInBinary(vP, t)
    
    # Schritt 3
    # This is wrong, search for Q first. len(Q) ~ len(P). Q not max
    ub = int(int(tempN)//int(P)) 
    lb = int((tempN - pow(2, MAX_PRIME) + 1) // P) 
    logger.info("Bounds: [{0}, {1}] \t P: {2} \t deltaB: {3}".format(lb,ub,P, ub-lb))
    R = 0
    Q = 0
    try:
        for i in range(MAX_ITERS):
            Q = int(genPrimeWithLimit(lb, ub))
            R = tempN - P*Q

            if (R < pow(2,MAX_PRIME+1) and R >= 0):
                valid = True
                break
            r_too_large += 1
    except ValueError:
        pass
    User["P"] = P
    User["Q"] = Q

Attacker["vP"] = vP
Attacker["R"] = R

logger.info("Amount of failed attempts: {0}".format(r_too_large))

logger.info("tempN = P * Q + R")
logger.info("{0} = {1} * {2} + {3}".format(tempN, P, Q, R))

INFO Bounds: [2, 4] 	 P: 431 	 deltaB: 2
INFO Amount of failed attempts: 0
INFO tempN = P * Q + R
INFO 1931 = 431 * 3 + 638


## Schritt 4

In [888]:
User["N"] = calcN(User["P"], User["Q"])

## Schritt 5

In [889]:
def createUserKey():
    PhiN = calcPhiN(P, Q)
    E = fermatPrime()
    D = mmi(E, PhiN)

    return  {
        "PhiN" : PhiN,
        "E" : E,
        "D" : D
    }

created = False
while not created:
    try:
        User.update(createUserKey())
        created = True
    except ValueError:
            logger.info("Trying to generate Key failed. Trying again ...")
logger.info("User Key successfully generated!")

INFO User Key successfully generated!


## Schritt 6

In [890]:
cipher = pow(msg, User["E"], User["N"])
signature = pow(msg, User["D"], User["N"])

# Angriff

## Schritt 1

In [891]:
pub_User = {
    "N" : User["N"],
    "E" : User["E"]
}

collection_User = {
    "N" : pub_User["N"],
    "E" : pub_User["E"]
}

## Schritt 2

In [892]:
collection_User["vP"] = splitInBinary(User["N"])[0]
collection_User["vP'"] = collection_User["vP"]+1

### Validation test for vP

In [893]:
testVar = "vP"
altTestVar = "vP'"

if(collection_User[testVar] == Attacker[testVar] or collection_User[altTestVar] == Attacker[testVar]):
    logger.info("Success!!!")
    logger.info("The variable {0} or {1} is in the expected state: {2}!".format(testVar, altTestVar, Attacker[testVar]))
    logger.info("Current {0}: {1} \t Current {2}: {3}".format(testVar, collection_User[testVar], altTestVar, collection_User[altTestVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} doesn't have the expected value.".format(testVar))
    logger.error("Expected: {0} \t Current: {1}".format(Attacker[testVar], collection_User[testVar]))

INFO Success!!!
INFO The variable vP or vP' is in the expected state: 1!
INFO Current vP: 1 	 Current vP': 2


## Schritt 3

In [899]:
collection_User["P"] = pow(collection_User["vP"], Attacker["D"], Attacker["N"])
collection_User["P'"] = pow(collection_User["vP'"], Attacker["D"], Attacker["N"])

print(collection_User, Attacker)

{'N': 1293, 'E': 3, 'vP': 1, "vP'": 2, 'P': 1, "P'": 8, 'Q': 1293, "Q'": 161, 'PhiN': 0, "PhiN'": 25600} {'P': 2, 'Q': 5, 'PhiN': 4, 'N': 10, 'E': 3, 'D': 3, 'vP': 1, 'R': 638}


### Validation test for P and P'

In [895]:
testVar = "P"
altTestVar = "P'"

if(collection_User[testVar] == User[testVar] or collection_User[altTestVar] == User[testVar]):
    logger.info("Success!!!")
    logger.info("The variables {0} or {1} are in the expected state: {2}!".format(testVar, altTestVar, User["P"]))
    logger.info("Current {0}: {1} \t Current {2}: {3}".format(testVar, collection_User[testVar], altTestVar, collection_User[altTestVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} or {1} doesn't have the expected value.".format(testVar, altTestVar))
    logger.error("Expected: {0} \t Current: {1} or {2}".format(User[testVar], collection_User[testVar], collection_User[altTestVar]))

ERROR Error!!!
ERROR The variable P or P' doesn't have the expected value.
ERROR Expected: 431 	 Current: 1 or 8


# Schritt 4

In [896]:
# Potential to optimize when N/P not hole Number
collection_User["Q"] = int(pub_User["N"]//collection_User["P"])
collection_User["Q'"] = int(pub_User["N"]//collection_User["P'"])

### Validation test for Q and Q'

In [897]:
testVar = "Q"
altTestVar = "Q'"

if(collection_User[testVar] == User[testVar] or collection_User[altTestVar] == User[testVar]):
    logger.info("Success!!!")
    logger.info("The variables {0} or {1} are in the expected state!".format(testVar, altTestVar))
    logger.info("Current {0}: {1} \t Current {2}: {3}".format(testVar, collection_User[testVar], altTestVar, collection_User[altTestVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} doesn't have the expected value.".format(testVar))
    logger.error("Expected: {0} \t Current: {1} or {2}".format(Attacker[testVar], collection_User[testVar], collection_User[altTestVar]))

ERROR Error!!!
ERROR The variable Q doesn't have the expected value.
ERROR Expected: 5 	 Current: 1293 or 161


## Schritt 5

In [898]:
collection_User["PhiN"] = int(calcPhiN(collection_User["P"], collection_User["Q"]))
collection_User["PhiN'"] = int(calcPhiN(collection_User["Q'"], collection_User["Q'"]))

collection_User["D"] = mmi(pub_User["E"], collection_User["PhiN"])
collection_User["D'"] = mmi(pub_User["E"], collection_User["PhiN'"])

ValueError: pow() 3rd argument cannot be 0

### Validation test for PhiN and PhiN'

In [ ]:
testVar = "PhiN"
altTestVar = "PhiN'"

if(collection_User[testVar] == User[testVar] or collection_User[altTestVar] == User[testVar]):
    logger.info("Success!!!")
    logger.info("The variables {0} or {1} are in the expected state!".format(testVar, altTestVar))
    logger.info("Current {0}: {1} \t Current {2}: {3}".format(testVar, collection_User[testVar], altTestVar, collection_User[altTestVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} doesn't have the expected value.".format(testVar))
    logger.error("Expected: {0} \t Current: {1} or {2}".format(Attacker[testVar], collection_User[testVar], collection_User[altTestVar]))

ERROR Error!!!
ERROR The variable PhiN doesn't have the expected value.
ERROR Expected: 280 	 Current: 227136 or 828100


### Validation test for D and D'

In [ ]:
testVar = "D"
altTestVar = "D'"

if(collection_User[testVar] == User[testVar] or collection_User[altTestVar] == User[testVar]):
    logger.info("Success!!!")
    logger.info("The variables {0} or {1} are in the expected state!".format(testVar, altTestVar))
    logger.info("Current {0}: {1} \t Current {2}: {3}".format(testVar, collection_User[testVar], altTestVar, collection_User[altTestVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} doesn't have the expected value.".format(testVar))
    logger.error("Expected: {0} \t Current: {1} or {2}".format(Attacker[testVar], collection_User[testVar], collection_User[altTestVar]))

ERROR Error!!!
ERROR The variable D doesn't have the expected value.
ERROR Expected: 73 	 Current: 152897 or 521993


## Schritt 6

In [ ]:
collection_User["Sig"] = pow(msg, collection_User["D"], User["N"])
collection_User["Sig'"] = pow(msg, collection_User["D'"], User["N"])

### Validation test for signature and signature'

In [ ]:
testVar = "Sig"
altTestVar = "Sig'"

if(collection_User[testVar] == signature):
    collection_User["correct_key"] = "D"
    logger.info("D is the right privat Key: {0}".format(collection_User[testVar]))
elif(collection_User[altTestVar] == signature):
    collection_User["correct_key"] = "D'"
    logger.info("D' is the right privat Key: {0}".format(collection_User[altTestVar]))
else:
    logger.error("Message was incorrectly signed!")

ERROR Message was incorrectly signed!


# Result

In [ ]:
def sortDic(dict):
    return {key: dict[key] for key in sorted(dict)}

In [ ]:
print(sortDic(Attacker))
print(sortDic(User))
print(sortDic(collection_User))



{'D': 73, 'E': 257, 'N': 319, 'P': 11, 'PhiN': 280, 'Q': 29, 'R': 978, 'vP': 224}
{'D': 12393, 'E': 257, 'N': 228661, 'P': 251, 'PhiN': 227500, 'Q': 911}
{'D': 152897, "D'": 521993, 'E': 257, 'N': 228661, 'P': 170, "P'": 251, 'PhiN': 227136, "PhiN'": 828100, 'Q': 1345, "Q'": 911, 'Sig': 226561, "Sig'": 44560, 'vP': 223, "vP'": 224}
